In [106]:
import random
import numpy as np
from collections import Counter

# Load a word list
def load_words(filename="words.txt", min_length=1):
    with open(filename) as f:
        words = [line.strip().lower() for line in f if len(line.strip()) >= min_length]
    return words

# Hangman Game Logic
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guessed = set()
        self.incorrect_guesses = set()
        self.state = ["_" if c.isalpha() else c for c in word]

    def guess(self, letter):
        print(f"Guessing letter: {letter}")
        self.guessed.add(letter)
        if letter in self.word:
            print(f"{letter} is in the word!")
            for i, c in enumerate(self.word):
                if c == letter:
                    self.state[i] = letter
        else:
            print(f"{letter} is NOT in the word.")
            self.incorrect_guesses.add(letter)

    def get_pattern(self):
        return "".join(self.state)

    def is_solved(self):
        return "_" not in self.state

    def allowed_guesses(self):
        return set("abcdefghijklmnopqrstuvwxyz") - self.guessed

class HybridStrategy:
    def __init__(self, words, alpha=0.3):
        self.dictionary = words
        self.alpha = alpha
        self.guessed_letters = set()  # Track guessed letters

    def filter_words(self, pattern, incorrect_guesses):
        print(f"Filtering words for pattern: {pattern} with incorrect guesses: {incorrect_guesses}")
        required_counts = Counter(pattern.replace("_", ""))

        def valid_word(word):
            if len(word) != len(pattern):
                return False
            if any(letter in word for letter in incorrect_guesses):
                return False
            if any(word.count(letter) > required_counts.get(letter, 0) for letter in required_counts):
                return False
            return all((p == "_" or p == w) for p, w in zip(pattern, word))

        filtered = [word for word in self.dictionary if valid_word(word)]
        print(f"Remaining possible words: {len(filtered)}")
        return filtered

    def most_frequent_letter(self, words, allowed):
        letter_counts = Counter()
        for word in words:
            unique_letters = set(word) - self.guessed_letters  # Consider only unguessed letters
            letter_counts.update(unique_letters)
        most_common = max((l for l in letter_counts if l in allowed), key=letter_counts.get, default=None)
        print(f"Most frequent letter: {most_common}")
        return most_common

    def best_information_gain(self, words, allowed):
        total_words = len(words)
        letter_probs = {l: sum(1 for w in words if l in w) / total_words for l in allowed}

        info_gain = {}
        for l, p in letter_probs.items():
            if p in [0, 1]:
                info_gain[l] = 0  # No information gained if probability is 0 or 1
            else:
                info_gain[l] = - (p * np.log2(p) + (1 - p) * np.log2(1 - p))

        best_letter = max(info_gain, key=info_gain.get, default=None)
        print(f"Best letter by information gain: {best_letter}")
        return best_letter

    def choose_letter(self, pattern, incorrect_guesses, allowed):
        filtered_words = self.filter_words(pattern, incorrect_guesses)
        if not filtered_words:
            guess = random.choice(list(allowed))
        else:
            common_letter = self.most_frequent_letter(filtered_words, allowed)
            if common_letter and sum(1 for w in filtered_words if common_letter in w) / len(filtered_words) > self.alpha:
                print(f"Choosing most common letter: {common_letter}")
                guess = common_letter
            else:
                best_info_letter = self.best_information_gain(filtered_words, allowed)
                print(f"Choosing best information gain letter: {best_info_letter}")
                guess = best_info_letter

        self.guessed_letters.add(guess)  # Track guessed letters
        return guess


# Running a simulation
def run_simulation(strategy, word):
    game = Hangman(word)
    print(f"Starting game with word: {word}")
    while not game.is_solved() and len(game.incorrect_guesses) < 6:
        guess = strategy.choose_letter(game.get_pattern(), game.incorrect_guesses, game.allowed_guesses())
        game.guess(guess)
        print(f"Current state: {game.get_pattern()} | Incorrect guesses: {game.incorrect_guesses}")
    print(f"Final state: {game.get_pattern()} | Solved: {game.is_solved()} | Total guesses: {len(game.guessed)}")
    return game.is_solved(), len(game.guessed), game.get_pattern()


In [99]:
### get all possible letters
words = load_words()
letters = set(letter for word in words for letter in word)

In [102]:
letters

{'!',
 '&',
 "'",
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '5',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
'''
so i'll use 37 letters and pad all words to length 50
i'll play a bunch of games and record my moves and the outcome of each game
'''

In [111]:
import random
import numpy as np
from collections import Counter
import cupy as cp
random.seed(42)

# English letter frequencies (for simplicity, you can use a predefined list)
ENGLISH_LETTER_FREQUENCY = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253, 'e': 0.12702,
    'f': 0.02228, 'g': 0.02015, 'h': 0.06094, 'i': 0.06966, 'j': 0.00153,
    'k': 0.00772, 'l': 0.04025, 'm': 0.02406, 'n': 0.06749, 'o': 0.07507,
    'p': 0.01929, 'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056,
    'u': 0.02758, 'v': 0.00978, 'w': 0.02360, 'x': 0.00150, 'y': 0.01974,
    'z': 0.00074
}

# Load a word list
def load_words(filename="words.txt", min_length=5):
    with open(filename) as f:
        words = [line.strip().lower() for line in f if len(line.strip()) >= min_length]
    return words

# Hangman Game Logic
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guessed = set()
        self.incorrect_guesses = set()
        self.state = ["_" if c.isalpha() else c for c in word]

    def guess(self, letter):
        self.guessed.add(letter)
        if letter in self.word:
            for i, c in enumerate(self.word):
                if c == letter:
                    self.state[i] = letter
        else:
            self.incorrect_guesses.add(letter)

    def get_pattern(self):
        return "".join(self.state)

    def is_solved(self):
        return "_" not in self.state

    def allowed_guesses(self):
        return set("abcdefghijklmnopqrstuvwxyz") - self.guessed

# Strategy Implementation
class HybridStrategy:
    def __init__(self, words, alpha=0.3, verbosity=1):
        self.dictionary = words
        self.alpha = alpha
        self.guessed_letters = set()  # Track guessed letters
        self.verbosity = verbosity  # Verbosity level (0 = silent, 1 = normal, 2 = debug)


    def sample_from_english_frequency(self, allowed):
        # Filter English frequencies to include only allowed letters
        allowed_frequencies = {l: ENGLISH_LETTER_FREQUENCY[l] for l in allowed if l in ENGLISH_LETTER_FREQUENCY}

        # If there are no allowed letters, return None
        if not allowed_frequencies:
            return None

        # Normalize the frequencies so they sum to 1
        total_freq = sum(allowed_frequencies.values())
        normalized_freqs = {l: f / total_freq for l, f in allowed_frequencies.items()}

        # Sample a letter based on the frequencies
        letter = random.choices(list(normalized_freqs.keys()), weights=normalized_freqs.values())[0]
        return letter


    def filter_words(self, pattern, incorrect_guesses):
        required_counts = Counter(pattern.replace("_", ""))

        def valid_word(word):
            if len(word) != len(pattern):
                return False
            if any(letter in word for letter in incorrect_guesses):
                return False
            if any(word.count(letter) > required_counts.get(letter, 0) for letter in required_counts):
                return False
            return all((p == "_" or p == w) for p, w in zip(pattern, word))

        filtered = [word for word in self.dictionary if valid_word(word)]
        if self.verbosity >= 2:
            print(f"Filtering words for pattern: {pattern} with incorrect guesses: {incorrect_guesses}")
            print(f"Remaining possible words: {len(filtered)}")
        return filtered

    def most_common_letter(self, words, allowed):
        letter_counts = Counter()
        for word in words:
            word_lets = [l for l in word if l not in self.guessed_letters]
            letter_counts.update(word_lets)
        most_common = max((l for l in letter_counts if l in allowed), key=letter_counts.get, default=None)
        if self.verbosity >= 2:
            print(f"Most frequent letter: {most_common}")
        # print(letter_counts)
        return most_common

    def most_frequent_letter(self, words, allowed):
        letter_counts = Counter()
        for word in words:
            unique_letters = set(word) - self.guessed_letters
            letter_counts.update(unique_letters)
        most_common = max((l for l in letter_counts if l in allowed), key=letter_counts.get, default=None)
        if self.verbosity >= 2:
            print(f"Most frequent letter: {most_common}")
        # print(letter_counts)
        return most_common

    def best_information_gain(self, words, allowed):
        total_words = len(words)
        letter_probs = {l: sum(1 for w in words if l in w) / total_words for l in allowed}

        info_gain = {}
        for l, p in letter_probs.items():
            if p in [0, 1]:
                info_gain[l] = 0  # No information gained if probability is 0 or 1
            else:
                info_gain[l] = - (p * np.log2(p) + (1 - p) * np.log2(1 - p))

        best_letter = max(info_gain, key=info_gain.get, default=None)
        if self.verbosity >= 2:
            print(f"Best letter by information gain: {best_letter}")
        # sorted_info_gain = sorted(info_gain.items(), key=lambda item: item[1], reverse=True)
        # print(sorted_info_gain)
        return best_letter, info_gain[best_letter]


    def choose_letter(self, pattern, incorrect_guesses, allowed):
        filtered_words = self.filter_words(pattern, incorrect_guesses)
        if not filtered_words:
            guess = random.choice(list(allowed))
        else:
            common_letter = self.most_frequent_letter(filtered_words, allowed)
            guess = common_letter
        self.guessed_letters.add(guess)
        if self.verbosity >= 1:
            print(f"Chosen letter: {guess}")
        return guess

# Running multiple games and recording the win rate
def run_simulation(strategy, words, num_games=1, val_words=None):
    wins = 0
    total_guesses = 0

    for i in range(num_games):
        if val_words is None:
            word = random.choice(words)
        else:
            word = random.choice(val_words)
        game = Hangman(word)
        strategy.guessed_letters.clear()  # Reset guesses per game

        if strategy.verbosity >= 1:
            print(f"\nGame {i+1}: Starting with word {word}")

        while not game.is_solved() and len(game.incorrect_guesses) < 6:
            guess = strategy.choose_letter(game.get_pattern(), game.incorrect_guesses, game.allowed_guesses())
            game.guess(guess)
            if strategy.verbosity >= 1:
                print(f"Current state: {game.get_pattern()} | Incorrect guesses: {len(game.incorrect_guesses)}")

        if game.is_solved():
            wins += 1
        total_guesses += len(game.guessed)

        if strategy.verbosity >= 1:
            print(f"Final state: {game.get_pattern()} | Solved: {game.is_solved()} | Total guesses: {len(game.guessed)}")

    win_rate = wins / num_games
    avg_guesses = total_guesses / num_games

    print(f"\nResults over {num_games} games:")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average guesses per game: {avg_guesses:.2f}")

    return win_rate, avg_guesses

if __name__ == "__main__":
  words = load_words()
  strategy = HybridStrategy(words, alpha=0.0, verbosity=1)
  num_games = 10  # Set the number of games to play
  run_simulation(strategy, words, num_games)



Game 1: Starting with word sane-minded
Chosen letter: e
Current state: ___e-____e_ | Incorrect guesses: 0
Chosen letter: d
Current state: ___e-___ded | Incorrect guesses: 0
Chosen letter: n
Current state: __ne-__nded | Incorrect guesses: 0
Chosen letter: s
Current state: s_ne-__nded | Incorrect guesses: 0
Chosen letter: i
Current state: s_ne-_inded | Incorrect guesses: 0
Chosen letter: m
Current state: s_ne-minded | Incorrect guesses: 0
Chosen letter: a
Current state: sane-minded | Incorrect guesses: 0
Final state: sane-minded | Solved: True | Total guesses: 7

Game 2: Starting with word carpic
Chosen letter: e
Current state: ______ | Incorrect guesses: 1
Chosen letter: a
Current state: _a____ | Incorrect guesses: 1
Chosen letter: i
Current state: _a__i_ | Incorrect guesses: 1
Chosen letter: n
Current state: _a__i_ | Incorrect guesses: 2
Chosen letter: s
Current state: _a__i_ | Incorrect guesses: 3
Chosen letter: r
Current state: _ar_i_ | Incorrect guesses: 3
Chosen letter: c
Current 

93% win rate with purely frequency based strategy
using only information is much worse
using alpha = 0.9 gives 86% win rate
92% with alpha=0.7

In [109]:
import random
import numpy as np
from collections import Counter, defaultdict

# Load a word list and split into training and validation sets
def load_words(filename="words.txt", min_length=5, split_ratio=0.9):
    with open(filename) as f:
        words = [line.strip().lower() for line in f if len(line.strip()) >= min_length]

    random.shuffle(words)
    split_idx = int(len(words) * split_ratio)
    training_set = words[:split_idx]
    validation_set = words[split_idx:]

    return training_set, validation_set

# Hangman Game Logic
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guessed = set()
        self.incorrect_guesses = set()
        self.state = ["_" if c.isalpha() else c for c in word]

    def guess(self, letter):
        self.guessed.add(letter)
        if letter in self.word:
            for i, c in enumerate(self.word):
                if c == letter:
                    self.state[i] = letter
        else:
            self.incorrect_guesses.add(letter)

    def get_pattern(self):
        return "".join(self.state)

    def is_solved(self):
        return "_" not in self.state

    def allowed_guesses(self):
        return set("abcdefghijklmnopqrstuvwxyz") - self.guessed

class NGramStrategy:
    def __init__(self, training_words, verbosity=1):
        self.dictionary = training_words
        self.guessed_letters = set()
        self.verbosity = verbosity
        # English letter frequency (most common to least common)
        self.letter_freq = list("etaoinsrhdlucmfywgpbvkjxqz")

        # Build n-gram index from training words for faster lookup
        self.ngram_index = self.build_ngram_index(training_words)

    def build_ngram_index(self, words):
        """Build an index of all n-grams in the training set for quick lookup."""
        ngram_index = defaultdict(list)
        for word in words:
            # Index all n-grams of length 2 or greater
            for n in range(2, len(word) + 1):
                for i in range(len(word) - n + 1):
                    ngram = word[i:i+n]
                    ngram_index[ngram].append((word, i))  # Store word and position
        return ngram_index

    def extract_largest_ngrams(self, pattern):
        """Extract the largest contiguous sequences where first and last characters are revealed."""
        ngrams = []
        current_ngram = ""

        # Scan for ngrams with revealed first and last characters
        for i in range(len(pattern)):
            if pattern[i] != "_":
                # Start a new ngram or continue current one
                if not current_ngram:
                    current_ngram = pattern[i]
                else:
                    current_ngram += pattern[i]
            else:
                # If we have an ngram and it has at least 2 characters
                if current_ngram and len(current_ngram) >= 2:
                    ngrams.append(current_ngram)
                current_ngram = ""  # Reset for next ngram

        # Don't forget the last ngram if we ended on a revealed character
        if current_ngram and len(current_ngram) >= 2:
            ngrams.append(current_ngram)

        # Also extract "partial" ngrams that have blanks in the middle
        i = 0
        while i < len(pattern):
            if pattern[i] != "_":  # Found a revealed letter
                # Look for another revealed letter ahead
                for j in range(i + 1, len(pattern)):
                    if pattern[j] != "_":  # Found another revealed letter
                        # Extract the substring with blanks
                        partial = pattern[i:j+1]
                        if partial.count("_") > 0 and partial not in ngrams:
                            ngrams.append(partial)
                i = j  # Skip to after the second revealed letter
            i += 1

        # Sort by length, longest first
        ngrams.sort(key=len, reverse=True)

        if self.verbosity >= 2:
            print(f"Extracted ngrams: {ngrams}")

        return ngrams

    def filter_words(self, pattern, incorrect_guesses):
        """Filter dictionary words based on the current pattern and incorrect guesses."""
        word_length = len(pattern)

        # Basic filtering: match length and exclude words with incorrect letters
        base_candidates = [word for word in self.dictionary
                           if len(word) == word_length and
                           not any(letter in word for letter in incorrect_guesses)]

        # Strict matching: pattern must exactly match at all non-underscore positions
        strict_matches = [word for word in base_candidates
                         if all((p == "_" or p == w) for p, w in zip(pattern, word))]

        return strict_matches

    def ngram_match_strategy(self, pattern, allowed):
        """Use the largest extracted n-grams to find likely letters."""
        letter_counts = Counter()
        word_length = len(pattern)

        # Extract the largest contiguous n-grams from the pattern
        ngrams = self.extract_largest_ngrams(pattern)

        if not ngrams:
            # If no useful n-grams, fall back to letter frequency
            return self.letter_frequency_strategy(allowed)

        # Find all dictionary words that contain any of our n-grams
        matching_words = []

        for ngram in ngrams:
            # Skip n-grams with underscores (partial matches)
            if "_" in ngram:
                continue

            # Look up this n-gram in our index
            matches = self.ngram_index.get(ngram, [])

            for word, position in matches:
                # Consider words of the right length
                if len(word) == word_length:
                    # Check if this n-gram appears at the same position in the pattern
                    pattern_pos = pattern.find(ngram)
                    if pattern_pos != -1 and pattern_pos == position:
                        matching_words.append(word)

        # Deduplicate matching words
        matching_words = list(set(matching_words))

        if self.verbosity >= 2:
            print(f"Found {len(matching_words)} words matching n-grams")

        # If we found matching words, analyze them
        if matching_words:
            # Count letter frequencies at underscore positions
            for word in matching_words:
                for i, c in enumerate(pattern):
                    if c == "_" and i < len(word):
                        letter = word[i]
                        if letter in allowed:
                            letter_counts[letter] += 1

            # If we found likely letters, choose the most common
            if letter_counts:
                return max(letter_counts, key=letter_counts.get)

        # Process partial n-grams (with blanks in the middle)
        for ngram in ngrams:
            if "_" in ngram:
                # Find the positions of underscores in this n-gram
                blank_positions = [i for i, c in enumerate(ngram) if c == "_"]

                # Find the position of this partial n-gram in the pattern
                pattern_pos = -1
                for i in range(len(pattern) - len(ngram) + 1):
                    if all(pattern[i+j] == ngram[j] for j in range(len(ngram)) if ngram[j] != "_"):
                        pattern_pos = i
                        break

                if pattern_pos != -1:
                    # Find words that match the non-blank parts of this n-gram
                    for word in self.dictionary:
                        if len(word) == word_length:
                            # Check if this word matches the non-blank parts at the right position
                            matches = True
                            for j in range(len(ngram)):
                                if ngram[j] != "_" and (pattern_pos + j >= len(word) or word[pattern_pos + j] != ngram[j]):
                                    matches = False
                                    break

                            if matches:
                                # If match found, count what letter fills each blank
                                for blank_pos in blank_positions:
                                    actual_pos = pattern_pos + blank_pos
                                    if actual_pos < len(word):
                                        letter = word[actual_pos]
                                        if letter in allowed:
                                            letter_counts[letter] += 1

            # If we found likely letters from partial n-grams, choose the most common
            if letter_counts:
                return max(letter_counts, key=letter_counts.get)

        # If all else fails, fall back to letter frequency
        return self.letter_frequency_strategy(allowed)

    def letter_frequency_strategy(self, allowed):
        """Fall back to general English letter frequency."""
        for letter in self.letter_freq:
            if letter in allowed:
                return letter
        return random.choice(list(allowed))

    def choose_letter(self, pattern, incorrect_guesses, allowed):
        """Main method to choose the next letter."""
        # First try dictionary-based approach
        filtered_words = self.filter_words(pattern, incorrect_guesses)

        if filtered_words:
            # Count letter frequencies in the filtered words
            letter_counts = Counter()
            for word in filtered_words:
                for letter in set(word) - self.guessed_letters:
                    if letter in allowed:
                        letter_counts[letter] += 1

            if letter_counts:
                guess = max(letter_counts, key=letter_counts.get)
                self.guessed_letters.add(guess)
                if self.verbosity >= 1:
                    print(f"Dictionary strategy: Selected {guess}")
                return guess

        # If dictionary approach fails, use n-gram matching
        guess = self.ngram_match_strategy(pattern, allowed)
        self.guessed_letters.add(guess)
        if self.verbosity >= 1:
            print(f"N-gram strategy: Selected {guess}")
        return guess



# Running multiple games and recording the win rate
def run_simulation(strategy, words, num_games=1):
    wins = 0
    total_guesses = 0

    for i in range(num_games):
        word = random.choice(words)
        game = Hangman(word)
        strategy.guessed_letters.clear()  # Reset guesses per game

        if strategy.verbosity >= 1:
            print(f"\nGame {i+1}: Starting with word {word}")

        while not game.is_solved() and len(game.incorrect_guesses) < 6:
            guess = strategy.choose_letter(game.get_pattern(), game.incorrect_guesses, game.allowed_guesses())
            game.guess(guess)
            if strategy.verbosity >= 1:
                print(f"Current state: {game.get_pattern()} | Incorrect guesses: {len(game.incorrect_guesses)}")

        if game.is_solved():
            wins += 1
        total_guesses += len(game.guessed)

        if strategy.verbosity >= 1:
            print(f"Final state: {game.get_pattern()} | Solved: {game.is_solved()} | Total guesses: {len(game.guessed)}")

    win_rate = wins / num_games
    avg_guesses = total_guesses / num_games

    print(f"\nResults over {num_games} games:")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average guesses per game: {avg_guesses:.2f}")

    return win_rate, avg_guesses

In [ ]:
if __name__ == "__main__":
    random.seed(42)
    training_words, validation_words = load_words()

    print(f"Training set size: {len(training_words)}")
    print(f"Validation set size: {len(validation_words)}")

    strategy = NGramStrategy(training_words, verbosity=1)

    num_validation_games = 1000  # Set number of validation games
    print("\nRunning validation set evaluation...")
    run_simulation(strategy, validation_words, num_validation_games)


Streaming output truncated to the last 5000 lines.
Dictionary strategy: Selected o
Current state: un_e___os_e__ | Incorrect guesses: 0
N-gram strategy: Selected l
Current state: un_e___os_e__ | Incorrect guesses: 1
N-gram strategy: Selected a
Current state: un_e___os_e__ | Incorrect guesses: 2
N-gram strategy: Selected i
Current state: un_e___os_e__ | Incorrect guesses: 3
N-gram strategy: Selected r
Current state: un_er_ros_e__ | Incorrect guesses: 3
N-gram strategy: Selected t
Current state: un_er_ros_e_t | Incorrect guesses: 3
N-gram strategy: Selected p
Current state: un_erprospe_t | Incorrect guesses: 3
N-gram strategy: Selected c
Current state: un_erprospect | Incorrect guesses: 3
N-gram strategy: Selected d
Current state: underprospect | Incorrect guesses: 3
Final state: underprospect | Solved: True | Total guesses: 13

Game 808: Starting with word amyloleucite
Dictionary strategy: Selected e
Current state: ______e____e | Incorrect guesses: 0
Dictionary strategy: Selected r
Curre

In [ ]:
'''
naive n gram approach gets about 39% on 100 games
on 1000 games it does: 33.5%

'''

In [123]:
import random
import numpy as np
from collections import Counter
import cupy as cp
import json
from pathlib import Path
import torch
from tqdm import tqdm

random.seed(42)

# English letter frequencies (for simplicity, you can use a predefined list)
ENGLISH_LETTER_FREQUENCY = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253, 'e': 0.12702,
    'f': 0.02228, 'g': 0.02015, 'h': 0.06094, 'i': 0.06966, 'j': 0.00153,
    'k': 0.00772, 'l': 0.04025, 'm': 0.02406, 'n': 0.06749, 'o': 0.07507,
    'p': 0.01929, 'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056,
    'u': 0.02758, 'v': 0.00978, 'w': 0.02360, 'x': 0.00150, 'y': 0.01974,
    'z': 0.00074
}

# Define vocabulary and create vocabulary index mapping
VOCABULARY = ['!', '&', "'", '-', '.', '/', '0', '1', '2', '3', '5',
              'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
              'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
              'w', 'x', 'y', 'z']
VOCAB_SIZE = len(VOCABULARY)
VOCAB_TO_IDX = {char: idx for idx, char in enumerate(VOCABULARY)}
IDX_TO_VOCAB = {idx: char for idx, char in enumerate(VOCABULARY)}

# Special token values
UNKNOWN_TOKEN = -1  # For unknown letters in the word ('_')
PADDING_TOKEN = -2  # For padding positions beyond the word length

# Load a word list
def load_words(filename="words.txt", min_length=5):
    with open(filename) as f:
        words = [line.strip().lower() for line in f if len(line.strip()) >= min_length]
    return words

# Game State Tracker
class GameStateTracker:
    def __init__(self, save_dir="game_states"):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True, parents=True)
        self.game_states = []
        self.current_game_id = 0

    def encode_state(self, pattern, incorrect_guesses, lives_left):
        # Encode pattern as a fixed-length vector (padded to length 50)
        # Each position contains:
        # - vocab index for revealed letters
        # - UNKNOWN_TOKEN (-1) for unknown letters ('_')
        # - PADDING_TOKEN (-2) for padding beyond word length
        pattern_vector = []
        for char in pattern:
            if char == '_':
                pattern_vector.append(UNKNOWN_TOKEN)  # Use -1 for unknown letters
            else:
                pattern_vector.append(VOCAB_TO_IDX.get(char, UNKNOWN_TOKEN))  # Get vocab index

        # Pad to length 50 with PADDING_TOKEN
        pattern_vector += [PADDING_TOKEN] * (50 - len(pattern_vector))

        # Encode incorrect guesses as a binary vector
        incorrect_guesses_vector = [0] * VOCAB_SIZE
        for letter in incorrect_guesses:
            if letter in VOCAB_TO_IDX:
                incorrect_guesses_vector[VOCAB_TO_IDX[letter]] = 1

        return {
            "pattern": pattern_vector,
            "incorrect_guesses": incorrect_guesses_vector,
            "lives_left": lives_left
        }

    def encode_action(self, letter):
        return VOCAB_TO_IDX.get(letter, -1)  # -1 if letter not in vocabulary

    def record_state_action(self, pattern, incorrect_guesses, lives_left, action_letter):
        game_state = self.encode_state(pattern, incorrect_guesses, lives_left)
        action_idx = self.encode_action(action_letter)

        state_action = {
            "game_id": self.current_game_id,
            "state": game_state,
            "action": action_idx
        }

        self.game_states.append(state_action)

    def start_new_game(self):
        self.current_game_id += 1

    def save_to_file(self, filename=None):
        if filename is None:
            filename = self.save_dir / f"game_states.json"

        with open(filename, 'w') as f:
            json.dump(self.game_states, f, indent=2)

        print(f"Saved {len(self.game_states)} state-action pairs to {filename}")

    def get_ml_tensors(self):
        """
        Convert the collected state-action pairs into PyTorch tensors for supervised learning.

        Returns:
            X: Tensor of shape (n_samples, feature_dim) containing state representations
            y: Tensor of shape (n_samples,) containing action labels
        """
        # Skip if no data
        if not self.game_states:
            return None, None

        # Extract features and targets
        X_list = []
        y_list = []

        for entry in self.game_states:
            # Create feature vector: pattern + incorrect_guesses + lives_left
            pattern = entry["state"]["pattern"]
            incorrect_guesses = entry["state"]["incorrect_guesses"]
            lives_left = entry["state"]["lives_left"]

            # Combine into one feature vector
            features = pattern + incorrect_guesses + [lives_left]
            X_list.append(features)

            # Target is the action
            y_list.append(entry["action"])

        # Convert to tensors
        X = torch.tensor(X_list, dtype=torch.float32)
        y = torch.tensor(y_list, dtype=torch.long)

        return X, y

    def save_ml_tensors(self, filename_prefix=None):
        """
        Save the state-action data as PyTorch tensors for supervised learning.
        """
        X, y = self.get_ml_tensors()

        if X is None or y is None:
            print("No data to save")
            return

        if filename_prefix is None:
            filename_prefix = self.save_dir / "hangman_ml"

        # Save tensors
        torch.save(X, f"{filename_prefix}_X.pt")
        torch.save(y, f"{filename_prefix}_y.pt")

        print(f"Saved tensors with shapes: X {X.shape}, y {y.shape}")
        print(f"Saved to {filename_prefix}_X.pt and {filename_prefix}_y.pt")

# Hangman Game Logic
class Hangman:
    def __init__(self, word, max_lives=6):
        self.word = word
        self.guessed = set()
        self.incorrect_guesses = set()
        self.state = ["_" if c.isalpha() else c for c in word]
        self.max_lives = max_lives

    def guess(self, letter):
        self.guessed.add(letter)
        if letter in self.word:
            for i, c in enumerate(self.word):
                if c == letter:
                    self.state[i] = letter
            return True
        else:
            self.incorrect_guesses.add(letter)
            return False

    def get_pattern(self):
        return "".join(self.state)

    def is_solved(self):
        return "_" not in self.state

    def allowed_guesses(self):
        return set("abcdefghijklmnopqrstuvwxyz") - self.guessed

    def lives_left(self):
        return self.max_lives - len(self.incorrect_guesses)

# Strategy Implementation
class HybridStrategy:
    def __init__(self, words, alpha=0.3, verbosity=1):
        self.dictionary = words
        self.alpha = alpha
        self.guessed_letters = set()  # Track guessed letters
        self.verbosity = verbosity  # Verbosity level (0 = silent, 1 = normal, 2 = debug)


    def sample_from_english_frequency(self, allowed):
        # Filter English frequencies to include only allowed letters
        allowed_frequencies = {l: ENGLISH_LETTER_FREQUENCY[l] for l in allowed if l in ENGLISH_LETTER_FREQUENCY}

        # If there are no allowed letters, return None
        if not allowed_frequencies:
            return None

        # Normalize the frequencies so they sum to 1
        total_freq = sum(allowed_frequencies.values())
        normalized_freqs = {l: f / total_freq for l, f in allowed_frequencies.items()}

        # Sample a letter based on the frequencies
        letter = random.choices(list(normalized_freqs.keys()), weights=normalized_freqs.values())[0]
        return letter


    def filter_words(self, pattern, incorrect_guesses):
        required_counts = Counter(pattern.replace("_", ""))

        def valid_word(word):
            if len(word) != len(pattern):
                return False
            if any(letter in word for letter in incorrect_guesses):
                return False
            if any(word.count(letter) > required_counts.get(letter, 0) for letter in required_counts):
                return False
            return all((p == "_" or p == w) for p, w in zip(pattern, word))

        filtered = [word for word in self.dictionary if valid_word(word)]
        if self.verbosity >= 2:
            print(f"Filtering words for pattern: {pattern} with incorrect guesses: {incorrect_guesses}")
            print(f"Remaining possible words: {len(filtered)}")
        return filtered

    def most_common_letter(self, words, allowed):
        letter_counts = Counter()
        for word in words:
            word_lets = [l for l in word if l not in self.guessed_letters]
            letter_counts.update(word_lets)
        most_common = max((l for l in letter_counts if l in allowed), key=letter_counts.get, default=None)
        if self.verbosity >= 2:
            print(f"Most frequent letter: {most_common}")
        # print(letter_counts)
        return most_common

    def most_frequent_letter(self, words, allowed):
        letter_counts = Counter()
        for word in words:
            unique_letters = set(word) - self.guessed_letters
            letter_counts.update(unique_letters)
        most_common = max((l for l in letter_counts if l in allowed), key=letter_counts.get, default=None)
        if self.verbosity >= 2:
            print(f"Most frequent letter: {most_common}")
        # print(letter_counts)
        return most_common

    def best_information_gain(self, words, allowed):
        total_words = len(words)
        letter_probs = {l: sum(1 for w in words if l in w) / total_words for l in allowed}

        info_gain = {}
        for l, p in letter_probs.items():
            if p in [0, 1]:
                info_gain[l] = 0  # No information gained if probability is 0 or 1
            else:
                info_gain[l] = - (p * np.log2(p) + (1 - p) * np.log2(1 - p))

        best_letter = max(info_gain, key=info_gain.get, default=None)
        if self.verbosity >= 2:
            print(f"Best letter by information gain: {best_letter}")
        # sorted_info_gain = sorted(info_gain.items(), key=lambda item: item[1], reverse=True)
        # print(sorted_info_gain)
        return best_letter, info_gain[best_letter]


    def choose_letter(self, pattern, incorrect_guesses, allowed):
        filtered_words = self.filter_words(pattern, incorrect_guesses)
        if not filtered_words:
            guess = random.choice(list(allowed))
        else:
            common_letter = self.most_frequent_letter(filtered_words, allowed)
            guess = common_letter
        self.guessed_letters.add(guess)
        if self.verbosity >= 1:
            print(f"Chosen letter: {guess}")
        return guess

# Running multiple games and recording the win rate
def run_simulation(strategy, words, num_games=1, val_words=None, state_tracker=None):
    wins = 0
    total_guesses = 0

    for i in tqdm(range(num_games)):
        if state_tracker:
            state_tracker.start_new_game()

        if val_words is None:
            word = random.choice(words)
        else:
            word = random.choice(val_words)
        game = Hangman(word)
        strategy.guessed_letters.clear()  # Reset guesses per game

        if strategy.verbosity >= 1:
            print(f"\nGame {i+1}: Starting with word {word}")

        while not game.is_solved() and len(game.incorrect_guesses) < 6:
            pattern = game.get_pattern()
            lives_left = game.lives_left()

            # Choose a letter using the strategy
            guess = strategy.choose_letter(pattern, game.incorrect_guesses, game.allowed_guesses())

            # Record the state and action before making the move
            if state_tracker:
                state_tracker.record_state_action(
                    pattern,
                    game.incorrect_guesses,
                    lives_left,
                    guess
                )

            # Make the guess
            game.guess(guess)

            if strategy.verbosity >= 1:
                print(f"Current state: {game.get_pattern()} | Incorrect guesses: {len(game.incorrect_guesses)}")

        if game.is_solved():
            wins += 1
        total_guesses += len(game.guessed)

        if strategy.verbosity >= 1:
            print(f"Final state: {game.get_pattern()} | Solved: {game.is_solved()} | Total guesses: {len(game.guessed)}")

    win_rate = wins / num_games
    avg_guesses = total_guesses / num_games

    print(f"\nResults over {num_games} games:")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average guesses per game: {avg_guesses:.2f}")

    if state_tracker:
        state_tracker.save_to_file()
        state_tracker.save_ml_tensors()

    return win_rate, avg_guesses

if __name__ == "__main__":
    words = load_words()
    strategy = HybridStrategy(words, alpha=0.0, verbosity=0)
    num_games = 100  # Set the number of games to play

    # Create a state tracker to record game states and actions
    state_tracker = GameStateTracker(save_dir="game_states")

    # Run the simulation with state tracking
    run_simulation(strategy, words, num_games=num_games, val_words=None, state_tracker=state_tracker)

100%|██████████| 100/100 [02:28<00:00,  1.48s/it]


Results over 100 games:
Win rate: 93.00%
Average guesses per game: 9.20
Saved 920 state-action pairs to game_states/game_states.json
Saved tensors with shapes: X torch.Size([920, 88]), y torch.Size([920])
Saved to game_states/hangman_ml_X.pt and game_states/hangman_ml_y.pt


In [113]:
X = torch.load('game_states/hangman_ml_X.pt')
y = torch.load('game_states/hangman_ml_y.pt')

In [114]:
X.shape

torch.Size([93, 88])

In [119]:
X[6]

tensor([29., -1., 24., 15.,  3., 23., 19., 24., 14., 15., 14., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.])

In [120]:
y[6]

tensor(11)

In [121]:
IDX_TO_VOCAB[11]

'a'

In [125]:
import random
import numpy as np
from collections import Counter
import json
from pathlib import Path
import torch
from tqdm import tqdm
import multiprocessing as mp
from functools import partial

# ... [Keep all existing classes and imports] ...

# New function to run a single game
def run_single_game(word, strategy_class, words_list, verbosity=0, state_tracker=None, game_id=0):
    strategy = strategy_class(words_list, alpha=0.0, verbosity=verbosity)
    game = Hangman(word)

    if state_tracker:
        state_tracker.current_game_id = game_id

    if verbosity >= 1:
        print(f"\nGame {game_id}: Starting with word {word}")

    guesses = 0

    while not game.is_solved() and len(game.incorrect_guesses) < 6:
        pattern = game.get_pattern()
        lives_left = game.lives_left()

        # Choose a letter using the strategy
        guess = strategy.choose_letter(pattern, game.incorrect_guesses, game.allowed_guesses())
        guesses += 1

        # Record the state and action before making the move
        if state_tracker:
            state_tracker.record_state_action(
                pattern,
                game.incorrect_guesses,
                lives_left,
                guess
            )

        # Make the guess
        game.guess(guess)

        if verbosity >= 1:
            print(f"Current state: {game.get_pattern()} | Incorrect guesses: {len(game.incorrect_guesses)}")

    if verbosity >= 1:
        print(f"Final state: {game.get_pattern()} | Solved: {game.is_solved()} | Total guesses: {guesses}")

    return {
        "won": game.is_solved(),
        "guesses": guesses,
        "word": word,
        "state_tracker": state_tracker
    }

# Parallelized simulation
def run_parallel_simulation(strategy_class, words, num_games=1000, val_words=None, num_processes=None, verbosity=0):
    if num_processes is None:
        num_processes = mp.cpu_count()  # Use all available cores

    print(f"Running simulation with {num_processes} processes")

    # Prepare the word list
    if val_words is None:
        game_words = [random.choice(words) for _ in range(num_games)]
    else:
        game_words = [random.choice(val_words) for _ in range(num_games)]

    # Create a partial function with fixed parameters
    game_func = partial(
        run_single_game,
        strategy_class=strategy_class,
        words_list=words,
        verbosity=verbosity
    )

    # Create argument tuples for each game
    args = [(word, i) for i, word in enumerate(game_words)]

    # Run games in parallel
    with mp.Pool(processes=num_processes) as pool:
        results = list(tqdm(pool.starmap(game_func, args), total=num_games))

    # Combine results
    wins = sum(r["won"] for r in results)
    total_guesses = sum(r["guesses"] for r in results)

    win_rate = wins / num_games
    avg_guesses = total_guesses / num_games

    print(f"\nResults over {num_games} games:")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average guesses per game: {avg_guesses:.2f}")

    return win_rate, avg_guesses, results

# Modified version that handles state tracking across processes
def run_parallel_simulation_with_tracking(strategy_class, words, num_games=1000, val_words=None, num_processes=None, verbosity=0):
    if num_processes is None:
        num_processes = mp.cpu_count()  # Use all available cores

    # For state tracking, we'll need to handle it differently since we can't easily share trackers between processes
    # Each process will create its own tracker and we'll combine them at the end

    # Split games into chunks for each process
    games_per_process = num_games // num_processes
    remaining_games = num_games % num_processes

    # Create process tasks
    tasks = []
    game_id_offset = 0

    for i in range(num_processes):
        # Determine number of games for this process
        process_games = games_per_process + (1 if i < remaining_games else 0)

        # Create a tuple with process info
        process_info = {
            "process_id": i,
            "num_games": process_games,
            "game_id_offset": game_id_offset,
            "save_dir": f"game_states_process_{i}"
        }

        tasks.append(process_info)
        game_id_offset += process_games

    # Run each process chunk
    with mp.Pool(processes=num_processes) as pool:
        process_results = list(pool.map(
            partial(
                run_process_chunk,
                strategy_class=strategy_class,
                words=words,
                val_words=val_words,
                verbosity=verbosity
            ),
            tasks
        ))

    # Combine results from all processes
    total_wins = sum(pr["wins"] for pr in process_results)
    total_guesses = sum(pr["total_guesses"] for pr in process_results)

    win_rate = total_wins / num_games
    avg_guesses = total_guesses / num_games

    # Combine state trackers' data
    combined_tracker = GameStateTracker(save_dir="game_states_combined")
    for pr in process_results:
        combined_tracker.game_states.extend(pr["game_states"])

    # Save combined results
    combined_tracker.save_to_file()
    combined_tracker.save_ml_tensors()

    print(f"\nResults over {num_games} games:")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print(f"Average guesses per game: {avg_guesses:.2f}")

    return win_rate, avg_guesses

def run_process_chunk(process_info, strategy_class, words, val_words=None, verbosity=0):
    process_id = process_info["process_id"]
    num_games = process_info["num_games"]
    game_id_offset = process_info["game_id_offset"]
    save_dir = process_info["save_dir"]

    # Create a state tracker for this process
    state_tracker = GameStateTracker(save_dir=save_dir)

    # Set random seed based on process ID for diversity
    random.seed(42 + process_id)

    wins = 0
    total_guesses = 0

    # Run games for this process
    for i in tqdm(range(num_games)):
        if val_words is None:
            word = random.choice(words)
        else:
            word = random.choice(val_words)

        game_id = game_id_offset + i
        strategy = strategy_class(words, alpha=0.0, verbosity=verbosity)
        strategy.guessed_letters.clear()  # Reset guesses per game
        game = Hangman(word)

        state_tracker.current_game_id = game_id

        while not game.is_solved() and len(game.incorrect_guesses) < 6:
            pattern = game.get_pattern()
            lives_left = game.lives_left()

            # Choose a letter using the strategy
            guess = strategy.choose_letter(pattern, game.incorrect_guesses, game.allowed_guesses())

            # Record the state and action before making the move
            state_tracker.record_state_action(
                pattern,
                game.incorrect_guesses,
                lives_left,
                guess
            )

            # Make the guess
            game.guess(guess)

        if game.is_solved():
            wins += 1
        total_guesses += len(game.guessed)

    # Save this process's data
    process_filename = f"{save_dir}/game_states_process_{process_id}.json"
    state_tracker.save_to_file(filename=process_filename)

    return {
        "process_id": process_id,
        "wins": wins,
        "total_guesses": total_guesses,
        "game_states": state_tracker.game_states  # Return the game states for combining
    }

if __name__ == "__main__":
    words = load_words()
    num_games = 100  # Increase number of games with parallelization
    num_processes = mp.cpu_count()  # Use all available CPU cores

    print(f"Running {num_games} games using {num_processes} processes")

    # Run the parallelized simulation with state tracking
    run_parallel_simulation_with_tracking(
        HybridStrategy,
        words,
        num_games=num_games,
        val_words=None,
        num_processes=num_processes,
        verbosity=0
    )

Running 100 games using 2 processes


100%|██████████| 50/50 [02:10<00:00,  2.60s/it]


Saved 476 state-action pairs to game_states_process_1/game_states_process_1.json


100%|██████████| 50/50 [02:10<00:00,  2.61s/it]


Saved 449 state-action pairs to game_states_process_0/game_states_process_0.json
Saved 925 state-action pairs to game_states_combined/game_states.json
Saved tensors with shapes: X torch.Size([925, 88]), y torch.Size([925])
Saved to game_states_combined/hangman_ml_X.pt and game_states_combined/hangman_ml_y.pt

Results over 100 games:
Win rate: 95.00%
Average guesses per game: 9.25


In [127]:
X = torch.load('game_states_combined/hangman_ml_X.pt')
y = torch.load('game_states_combined/hangman_ml_y.pt')

In [128]:
X.shape

torch.Size([925, 88])

In [129]:
X[0]

tensor([-1., -1., -1., -1.,  3., -1., -1., -1., -1., -1., -1., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.])

In [130]:
X[5]

tensor([29., -1., 24., 15.,  3., -1., 19., 24., 14., 15., 14., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.])

In [131]:
y[5]

tensor(23)

In [132]:
X[6]

tensor([29., -1., 24., 15.,  3., 23., 19., 24., 14., 15., 14., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.])

In [133]:
y[6]

tensor(11)

In [134]:
X[7]

tensor([-1., -1., -1., -1., -1., -1., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.])

In [135]:
y[7]

tensor(15)